In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Flatten,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
import os

In [3]:
base_model=VGG16(
    weights="vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5",
    include_top=False,
    input_shape=(224,224,3)
)

In [6]:
for layer in base_model.layers:
    layer.trainable=False

model=Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(10,activation='softmax')
])

In [7]:
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [9]:
data_dir="dataset"

train_datagen=ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator=train_datagen.flow_from_directory(
    data_dir,
    batch_size=32,
    target_size=(224,224),
    class_mode="categorical",
    subset="training"
)
val_generator=train_datagen.flow_from_directory(
    data_dir,
    batch_size=32,
    target_size=(224,224),
    class_mode="categorical",
    subset="validation"
)

Found 1000 images belonging to 10 classes.
Found 245 images belonging to 10 classes.


In [11]:
H=model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5
)


C:\Users\UTKARSH BRAHMANKAR\AppData\Roaming\Python\Python310\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 151s 5s/step - accuracy: 0.2145 - loss: 2.3791 - val_accuracy: 0.6531 - val_loss: 1.3448
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 156s 5s/step - accuracy: 0.6018 - loss: 1.5981 - val_accuracy: 0.6531 - val_loss: 1.2202
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 157s 5s/step - accuracy: 0.6297 - loss: 1.4077 - val_accuracy: 0.6612 - val_loss: 1.1287
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 155s 5s/step - accuracy: 0.6600 - loss: 1.2696 - val_accuracy: 0.6735 - val_loss: 1.0197
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 164s 5s/step - accuracy: 0.6998 - loss: 1.1045 - val_accuracy: 0.6898 - val_loss: 0.9630


In [12]:
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Re-compile with smaller learning rate
model.compile(optimizer=Adam(learning_rate=1e-10),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print("🔄 Fine-tuning top layers...")

fine_tune_history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5
)


🔄 Fine-tuning top layers...
Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 182s 6s/step - accuracy: 0.6514 - loss: 1.1564 - val_accuracy: 0.7020 - val_loss: 0.9483
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 179s 6s/step - accuracy: 0.7197 - loss: 1.0113 - val_accuracy: 0.7020 - val_loss: 0.9691
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 175s 5s/step - accuracy: 0.6773 - loss: 1.1244 - val_accuracy: 0.6939 - val_loss: 0.9552
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 172s 5s/step - accuracy: 0.6555 - loss: 1.1274 - val_accuracy: 0.6980 - val_loss: 0.9720
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 173s 5s/step - accuracy: 0.7017 - loss: 1.0716 - val_accuracy: 0.7061 - val_loss: 0.9616


In [13]:
# ==========================================================
# Step 8: Evaluate and Save the model
# ==========================================================
loss, acc = model.evaluate(val_generator)
print(f"\n✅ Final Validation Accuracy: {acc*100:.2f}%")

model.save("vgg16_transfer_learning_final.h5")
print("✅ Model saved successfully as vgg16_transfer_learning_final.h5")

8/8 ━━━━━━━━━━━━━━━━━━━━ 30s 4s/step - accuracy: 0.6974 - loss: 0.9756



✅ Final Validation Accuracy: 70.20%
✅ Model saved successfully as vgg16_transfer_learning_final.h5
